In [ ]:
# %pip install pinecone-client
# %pip install python-dotenv
# %pip install openai
# %pip install chromadb-client

# %pip install llama-index-vector-stores-chroma

In [ ]:
from openai import OpenAI, AzureOpenAI
import os
from dotenv import load_dotenv

load_dotenv(dotenv_path=".env")

In [ ]:
from pinecone import Pinecone

# pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])
# index = pc.Index("test-pdf")

In [ ]:
# from openai import OpenAI
# client = OpenAI()

# response = client.embeddings.create(
#     input="Your text string goes here",
#     model="text-embedding-3-large"
# )

# print(response.data[0].embedding)

In [ ]:
client = AzureOpenAI(
  api_key = os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version = "2024-02-01",
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)

response = client.embeddings.create(
    input="Your text string goes here",
    model="ada_gcal"
)

print(response.data[0].embedding)

In [ ]:
def generate_embeddings(text, model="ada_gcal"): # model = "deployment_name"
    return client.embeddings.create(input = [text], model=model).data[0].embedding

# df_bills['ada_v2'] = df_bills["text"].apply(lambda x : generate_embeddings (x, model = 'text-embedding-ada-002'))

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("./pdf/grokking.pdf")
pages = loader.load_and_split()

In [ ]:
print(len(pages))
print()

full_document = ""

for page in pages:
  full_document += page.page_content

print(full_document)

In [ ]:
import re

# first, extract Section Headers
pattern = r"\n\d+ [A-Z][a-zA-Z\s:]+\n"

# Find all matches
matches = re.findall(pattern, full_document)

matches.insert(0, "Abstract")

# Print all section titles
for match in matches:
    print(match.strip())
    print()

In [ ]:
pattern = r"\n\d+ [A-Z][a-zA-Z\s:]+\n"

# Split the text into sections
sections = re.split(pattern, full_document)

# number of sections should match with number of section titles (aka matches of regex pattern)
# print(sections[0])
# print(sections[2])
# print(len(sections))

# Create a dictionary to store section titles and contents
section_contents = {}

# Use zip to iterate over matches and sections simultaneously
for match, section in zip(matches, sections):
    section_title = match.strip()
    content = section.strip()
    section_contents[section_title] = content

section_contents.keys()


In [ ]:
section_contents["6 Conclusion"] #TODO: process this to ONLY the abstract content

In [ ]:
for title, content in section_contents.items():
    print(title)
    print(len(content))
    print()

In [ ]:
subsection = section_contents["3 Why Generalization Occurs: Representations and Dynamics"]
subsection

In [ ]:
# then extract subsections from sections
pattern = r"\n\d+\.\d+ [A-Z][a-zA-Z\s:]+\n"

subsections = re.findall(pattern, subsection)
print(len(subsections))
for match in subsections:
    print(match.strip())
    print()

In [105]:
# extract references from conclusion
abstract = section_contents["Abstract"]
section_contents["Abstract"] = abstract.split("Abstract")[1]

section_contents["Abstract"]

'\nWe aim to understand grokking , a phenomenon where models generalize long after\noverﬁtting their training set. We present both a microscopic analysis anchored by an\neffective theory and a macroscopic analysis of phase diagrams describing learning\nperformance across hyperparameters. We ﬁnd that generalization originates from\nstructured representations whose training dynamics and dependence on training set\nsize can be predicted by our effective theory in a toy setting. We observe empirically\nthe presence of four learning phases: comprehension ,grokking ,memorization , and\nconfusion . We ﬁnd representation learning to occur only in a “Goldilocks zone”\n(including comprehension and grokking) between memorization and confusion.\nWe ﬁnd on transformers the grokking phase stays closer to the memorization phase\n(compared to the comprehension phase), leading to delayed generalization. The\nGoldilocks phase is reminiscent of “intelligence from starvation” in Darwinian\nevolution, wher

In [ ]:
# extract references from conclusion
conclusion = section_contents["6 Conclusion"]
section_contents["6 Conclusion"] = conclusion.split("References")[0]
# conclusion.split("References")[1]

section_contents["6 Conclusion"]

In [ ]:
# run subsection splitting algo

for title, section in section_contents.items():
  pattern = r"\n\d+\.\d+ [A-Z][a-zA-Z\s:]+\n"
  subsections_headers = re.findall(pattern, section)
  
  if len(subsections_headers) > 0:
    subsection_map = {}
    subsections_content = re.split(pattern, section)

    subsections_headers.insert(0, title)
    for subsection_header, subsection in zip(subsections_headers, subsections_content):
      subsection_title = subsection_header.strip()
      content = subsection.strip()
      subsection_map[subsection_title] = content

    print(subsection_map.keys())
    section_contents[title] = subsection_map
    

In [ ]:
len(section_contents["3 Why Generalization Occurs: Representations and Dynamics"])
len(section_contents["4 Delayed Generalization: A Phase Diagram"])

In [ ]:
# use this for section title-based splitting
section_contents["4 Delayed Generalization: A Phase Diagram"]['4.1 Phase diagram of a toy model']

In [ ]:
def generate_embeddings(text, model="ada_gcal"): # model = "deployment_name"
    return client.embeddings.create(input = [text], model=model).data[0].embedding

# df_bills['ada_v2'] = df_bills["text"].apply(lambda x : generate_embeddings (x, model = 'text-embedding-ada-002'))

In [154]:
import chromadb.utils.embedding_functions as embedding_functions
azure_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key = os.getenv("AZURE_OPENAI_API_KEY"),  
                api_base=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_type="azure",
                api_version = "2024-02-01",
                model_name="ada_gcal"
            )

In [166]:
import chromadb
# Example setup of the client to connect to your chroma server
client = chromadb.HttpClient(host='localhost', port=8000)

In [167]:
# client.delete_collection(name="test_pdf_2") 
# Delete a collection and all associated embeddings, documents, and metadata. ⚠️ This is destructive and not reversible

In [168]:
collection = client.create_collection(
        embedding_function=azure_ef,
        name="test_pdf_2",
        metadata={"hnsw:space": "l2"} # l2 is the default
    )
collection = client.get_collection(name="test_pdf_2", embedding_function=azure_ef)

In [169]:
collection.peek() # returns a list of the first 10 items in the collection
# collection.count() # returns the number of items in the collection
# collection.modify(name="new_name") # Rename the collection

{'ids': [],
 'embeddings': [],
 'metadatas': [],
 'documents': [],
 'data': None,
 'uris': None}

In [170]:
# collection.add(
#     documents=["lorem ipsum...", "doc2", "doc3", ...],
#     metadatas=[{"chapter": "3", "verse": "16"}, {"chapter": "3", "verse": "5"}, {"chapter": "29", "verse": "11"}, ...],
#     ids=["id1", "id2", "id3", ...]
# )

import uuid

for title, content in section_contents.items():
  unique_id = str(uuid.uuid4())
  if type(content) == dict:
    for t, c in content.items():
      collection.add(
      documents = [c],
      metadatas=[{"section_title": t}],
      ids=[unique_id]
    )
  else:
    collection.add(
      documents = [content],
      metadatas=[{"section_title": title}],
      ids=[unique_id]
    )

In [171]:
multiQueryPrompt = """You are an AI language model assistant. Your task is to generate Five
    different versions of the given user question to retrieve relevant documents from a vector
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search.
    Provide these alternative questions seperated by newlines only.

    For example:
    User question: "What is the conclusion of the paper?"

    Generated questions:
    What is the main takeaway from the paper?
    What are the key findings of the paper?
    What is the summary of the paper?
    What is the final thought of the paper?
    What is the ending of the paper?
    
    Output format should be as follows:

    'What is Bill Gates known for?'
│   "Can you provide information about Bill Gates' background?"

    And not this format:

    '1. What is Bill Gates known for?'
│   "2. Can you provide information about Bill Gates' background?"
    """

In [172]:
client = AzureOpenAI(
  api_key = os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version = "2024-02-01",
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)

response = client.chat.completions.create(
    model="cursor-gpt-4", # model = "deployment_name".
    messages=[
        {"role": "system", "content": multiQueryPrompt},
        {"role": "user", "content": "why does generalization occur?"}
    ]
)

#print(response)
print(response.model_dump_json(indent=2))
print(response.choices[0].message.content)

questions = response.choices[0].message.content.split("\n")
print(questions)

{
  "id": "chatcmpl-9F8CPb7XKMd3GAlugZAiLwvaPwBFr",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "What is the reason behind generalization happening?\nWhat causes generalization?\nWhat are the principles responsible for generalization?\nWhat triggers the occurrence of generalization?\nWhat is the underlying mechanism causing generalization?",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "created": 1713393465,
  "model": "gpt-4-32k",
  "object": 

In [173]:
relevant_docs = collection.query(
    query_texts=questions,
    n_results=1,
    # where={"metadata_field": "is_equal_to_this"},
    # where_document={"$contains":"search_string"}
)

len(relevant_docs['documents'])

5

In [174]:
provided_context = relevant_docs
provided_context

{'ids': [['502463a7-4688-4dd0-9d6f-9a54b0add4b7'],
  ['502463a7-4688-4dd0-9d6f-9a54b0add4b7'],
  ['db9b51c0-a91e-4d1a-8042-4f5e21d308d5'],
  ['502463a7-4688-4dd0-9d6f-9a54b0add4b7'],
  ['502463a7-4688-4dd0-9d6f-9a54b0add4b7']],
 'distances': [[0.32639008158242083],
  [0.34207177947302075],
  [0.3684194708813717],
  [0.3392694363749785],
  [0.33198065226081047]],
 'embeddings': None,
 'metadatas': [[{'section_title': '1 Introduction'}],
  [{'section_title': '1 Introduction'}],
  [{'section_title': '3 Why Generalization Occurs: Representations and Dynamics'}],
  [{'section_title': '1 Introduction'}],
  [{'section_title': '1 Introduction'}]],
 'documents': [['Perhaps thecentral challenge of a scientiﬁc understanding of deep learning lies in accounting for neu-\nral network generalization. Power et al. [ 1] recently added a new puzzle to the task of understanding\ngeneralization with their discovery of grokking . Grokking refers to the surprising phenomenon of\ndelayed generalization where

In [175]:
class PromptTemplate:
    def __init__(self, input_variables, template):
        self.input_variables = input_variables
        self.template = template

    def format(self, **kwargs):
        return self.template.format(**kwargs)

In [184]:
# qaPrompt = """Here is the question you need to answer:

#     \n --- \n {query_str} \n --- \n

#     Here is any available background question + answer pairs:

#     \n --- \n {q_a_pairs} \n --- \n

#     Here is additional context relevant to the question:

#     \n --- \n {context_str} \n --- \n

#     Use the above context and any background question + answer pairs to answer the question: \n {query_str}
#     """

# TODO: q_a_pairs from FT-ed model

qaPrompt = """
Your task is to use the document excerpts to provide answers. 
Answer questions precisely and succinctly but provide full lists of steps and stipulations if the response requires it.
If you cannot provide answers based on the context provided, answer with "I cannot answer the question based on the context provided".

Answer questions with as much detail as possible. Think logically and take it step by step. If the content explains where to find more information, please include that in your answer.

DO NOT provide information other than what you have in the CONTENT.
Do NOT mention "the document" or "the context" in your answer.

Here is the document relevant to the question:

    ###{context_str}###

Do not create new context or use external sources to provide answers. Use only the context provided to answer the question.
    """

prompt_template = PromptTemplate(input_variables=["context_str"], template=qaPrompt)

In [185]:
formatted_prompt = prompt_template.format(context_str="only PyPDF library can be used to access Large Language Models")
print(formatted_prompt)


Your task is to use the document excerpts to provide answers. 
Answer questions precisely and succinctly but provide full lists of steps and stipulations if the response requires it.
If you cannot provide answers based on the context provided, answer with "I cannot answer the question based on the context provided".

Answer questions with as much detail as possible. Think logically and take it step by step. If the content explains where to find more information, please include that in your answer.

DO NOT provide information other than what you have in the CONTENT.
Do NOT mention "the document" or "the context" in your answer.

Here is the document relevant to the question:

    ###only PyPDF library can be used to access Large Language Models###

Do not create new context or use external sources to provide answers. Use only the context provided to answer the question.
    


In [187]:
uesr_query = "What libraries can be used to via Large Language Models?"

response = client.chat.completions.create(
    model="cursor-gpt-4", # model = "deployment_name".
    messages=[
        {"role": "system", "content": formatted_prompt},
        {"role": "user", "content": uesr_query}
    ]
)

#print(response)
print(response.model_dump_json(indent=2))
print(response.choices[0].message.content)


{
  "id": "chatcmpl-9FCZ3o1GaFN8nJl5bQi1XhvbP1rM4",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Only the PyPDF library can be used to access Large Language Models.",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "created": 1713410245,
  "model": "gpt-4-32k",
  "object": "chat.completion",
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 14,
    "prompt_tokens": 194,
    "total_tokens": 208
  },
  "prompt_filter_results": [
  